# Social Media Monitoring


The following table contains data on the politicians and their associated social acounts

In [ ]:
import os
from smm_wrapper import SMM
import qgrid
smm = SMM()

politicians_df = smm.dv.politicians_df()
qgrid.show_grid(politicians_df)

In [ ]:
#from collections import defaultdict 
#aggregating twitter data by party affiliation
party_list = []
for name, group in politicians_df.groupby(['affiliation']):
    party_list.append({'party':name,'labels':[],'values':[]})

def search(party, party_list):
    for element in party_list:
        if element['party'] == party:
            return dict(element)

for index, row in politicians_df.iterrows():
    if row['tw_ids'] is not None:
        tw = smm.api.tweets_by(twitter_user_id=row['tw_ids'][0], aggregate_by='month')
        for i, date in enumerate(tw['labels'], 0):
            if date in search(row['affiliation'], party_list)['labels']:
                curr_i = search(row['affiliation'], party_list)['labels'].index(date)
                search(row['affiliation'], party_list)['values'][curr_i] += tw['values'][i]
            else:
                search(row['affiliation'], party_list)['labels'].append(date)
                search(row['affiliation'], party_list)['values'].append(tw['values'][i])

In [ ]:
#plotting data
import plotly
from plotly import graph_objs
trace=[]
for element in party_list:
    dates = element['labels']
    values = element['values']
    data = sorted(zip(dates,values))
    x, y = zip(*data)
    trace.append(graph_objs.Scatter(x=x, y=y, mode='lines+markers', name=element['party']))

layout = graph_objs.Layout(hovermode='closest', xaxis=dict(title='Date'), yaxis=dict(title='Tweets'),showlegend=True)

plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({"data": trace, "layout": layout})